## 特徴量生成
### 学習に必要な列を抽出して集計
ここではデバイスごとに各センサー平均値と**RUL**(耐用期間)の関係を作成します。

In [34]:
pydf = spark.sql("""
    SELECT
        DeviceId,
        Period,
        max(Cycle) AS RUL,
        round(avg(Sensor11),2) AS avgSensor11,
        round(avg(Sensor14),2) AS avgSensor14,
        round(avg(Sensor15),2) AS avgSensor15,
        round(avg(Sensor9),2) AS avgSensor9
    FROM 
        sensortablespark
    WHERE
        endofperiod = 1 
    GROUP BY 
        DeviceId,
        Period
    """)
pydf.show(10)

+---------+------+---+-----------+-----------+-----------+----------+
| DeviceId|Period|RUL|avgSensor11|avgSensor14|avgSensor15|avgSensor9|
+---------+------+---+-----------+-----------+-----------+----------+
|N1172FJ-2|    16|172|      46.17|    8121.61|       8.78|   8782.62|
|N3172FJ-1|     5|164|      44.49|     8079.9|       9.27|   8748.87|
|N1172FJ-1|    52|149|      45.87|    8122.38|       8.75|    8778.9|
|N1172FJ-1|    35|134|      46.17|    8106.67|       8.72|    8764.4|
|N3172FJ-1|    24|203|      44.55|    8068.05|       9.21|   8728.73|
|N4172FJ-1|     6|168|      41.91|    8101.64|       9.32|   8338.59|
|N1172FJ-1|    25|228|      45.49|     8105.7|       8.91|    8720.3|
|N4172FJ-2|    15|177|       42.5|    8102.25|       9.43|   8346.38|
|N1172FJ-2|    46|277|      42.98|    8080.73|       9.47|   8331.11|
|N3172FJ-2|    48|242|      42.32|    8156.91|       9.37|   8396.64|
+---------+------+---+-----------+-----------+-----------+----------+
only showing top 10 

In [35]:
# Shape
print((pydf.count(), len(pydf.columns)))

(436, 7)

## 特徴量変換
SparkMLが実行可能なベクター形式に変換します

In [36]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
# 特徴量変換
vectorAssembler = VectorAssembler(inputCols = ['avgSensor11','avgSensor14','avgSensor15','avgSensor9'], outputCol = 'features')
# T-SQL PREDICT単純化のために特徴量削減ver
# vectorAssembler = VectorAssembler(inputCols = ['Sensor11'], outputCol = 'features')

vdf = vectorAssembler.transform(pydf)

## データセット分割


In [38]:
trainingFraction = 0.7
testingFraction = (1-trainingFraction)
seed = 42

# Split the dataframe into test and training dataframes
df_train, df_test = vdf.randomSplit([trainingFraction, testingFraction], seed=seed)

## モデル学習
線形回帰を利用しています。
実運用には時系列データ用の適切なアルゴリズムを選定ください。

In [42]:
# モデル作成
lin_reg = LinearRegression(featuresCol = 'features', labelCol='RUL', maxIter = 10, regParam=0.3)
model = lin_reg.fit(df_train)
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [48.681536516326126,0.7054508941857961,229.12063313718622,-0.13949520207323793]
Intercept: -8588.624013955796

## 推論結果の確認


In [44]:
# テストデータセットで推論実行
prediction = model.transform(df_test)

In [47]:
# 確認
display(prediction.select("features","RUL","prediction"))

## SQL Poolへのロード
明細としてのSparkテーブルと、最新のRUL予測値をSQL poolへロードします。

sqlanalyticsAPIを利用して簡単にロード可能です。

In [48]:
%%spark
val sqlDF = spark.sql("SELECT * FROM sensortablespark")

sqlDF: org.apache.spark.sql.DataFrame = [Cycle: bigint, DeviceId: string ... 9 more fields]

In [49]:
%%spark
import org.apache.spark.sql.SqlAnalyticsConnector._
import com.microsoft.spark.sqlanalytics.utils.Constants

val sql_pool_name = "aiaddw" //SQL pool 名
//SQL Poolへロード
sqlDF.write.sqlanalytics(s"$sql_pool_name.dbo.Sensor", Constants.INTERNAL)

import org.apache.spark.sql.SqlAnalyticsConnector._
import com.microsoft.spark.sqlanalytics.utils.Constants
sql_pool_name: String = aiaddw

## 最新日付のセンサー平均値集計


In [50]:
aggdf = spark.sql("""
    SELECT
        DeviceId,
        date_jst,
        round(avg(Sensor11),2) AS avgSensor11,
        round(avg(Sensor14),2) AS avgSensor14,
        round(avg(Sensor15),2) AS avgSensor15,
        round(avg(Sensor9),2) AS avgSensor9
    FROM 
        sensortablespark
    WHERE
        date_jst = (select max(date_jst) from sensortablespark)
    GROUP BY 
        DeviceId,date_jst
    """)

## 推論実行


In [52]:
#推論用前処理
vdf2 = vectorAssembler.transform(aggdf)

#スコアリング実行
predictdf = model.transform(vdf2)\
    .drop("features")\
    .withColumnRenamed("prediction","RUL")

#確認
predictdf.show()

#scalaへの受け渡し用tempview
predictdf.createOrReplaceTempView("tempPredict") 

+---------+----------+-----------+-----------+-----------+----------+------------------+
| DeviceId|  date_jst|avgSensor11|avgSensor14|avgSensor15|avgSensor9|               RUL|
+---------+----------+-----------+-----------+-----------+----------+------------------+
|N3172FJ-1|2020-04-30|      42.89|    8064.59|       9.31|   8520.16|133.09096758214582|
|N1172FJ-2|2020-04-30|      42.97|    8062.41|       9.34|   8518.08|  142.611376568555|
|N2172FJ-1|2020-04-30|      43.02|    8066.46|       9.33|   8525.53| 144.5720839290043|
|N3172FJ-2|2020-04-30|      42.92|    8065.31|        9.3|   8523.92| 132.2436300302852|
|N4172FJ-2|2020-04-30|      42.85|    8062.16|       9.32|   8514.89|132.45580649491785|
|N1172FJ-1|2020-04-30|      43.02|    8066.88|       9.33|   8526.77|144.69539925399476|
|N4172FJ-1|2020-04-30|      42.88|    8061.86|       9.32|   8518.24|133.23730839520795|
|N2172FJ-2|2020-04-30|      42.96|    8063.93|       9.34|   8518.53|143.13407372162146|
+---------+----------

## SQL Poolへのロード②


In [28]:
%%spark
var sql_pdf = spark.sql("select * from tempPredict")

sql_pdf: org.apache.spark.sql.DataFrame = [DeviceId: string, date_jst: date ... 5 more fields]

In [53]:
%%spark
//SQL Poolへロード
sql_pdf.write.sqlanalytics(s"$sql_pool_name.dbo.PREDICT_SensorRUL", Constants.INTERNAL)